In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from utils.imports import *
from tensorflow.keras.layers import *
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1" # "0, 1" for multiple

## Normalization

In [5]:
from data_reader import calc_mean_std_par_dir

In [7]:
subdir = '../data/SRAD2018_TRAIN_001/RAD_206482464212530/'

In [15]:
par_dir = Path('../data/SRAD2018_TRAIN_001')
subdirs = [o for o in par_dir.iterdir() if o.is_dir()]

In [ ]:
subdir

In [22]:
%%time
means, stds = zip(*[calc_mean_std_subdir(o) for o in subdirs])

In [23]:
means, stds

((0.9820557,
  0.7564683,
  0.968596,
  0.9650422,
  0.98231244,
  0.7932429,
  0.9655625,
  0.97146374,
  0.9070648,
  0.7613075),
 (0.12885971,
  0.3993739,
  0.16909033,
  0.17655694,
  0.12847538,
  0.39483172,
  0.17846707,
  0.16026804,
  0.28292444,
  0.41495243))

In [6]:
mean , std = np.mean(stds), np.mean(means)

NameError: name 'stds' is not defined

In [7]:
mean, std = calc_mean_std_par_dir('../data/SRAD2018_TRAIN_002')

KeyboardInterrupt: 

In [ ]:
mean, std

In [39]:
fn = 'stat.csv'
with open(fn, 'w') as f:
        np.array([mean, std]).tofile(f, ',')

In [40]:
!cat stat.csv

0.27102503180503845,0.8787143230438232

## SequenceGenerator

In [11]:
sess = tf.InteractiveSession()

In [6]:
shape = (61, 501, 501, 3)
def parser_train(serialized_example):
    features, sequence_features = tf.parse_single_sequence_example(
        serialized_example, context_features={
            'time_stamp': tf.FixedLenFeature([], tf.string),
        }, sequence_features={
            "raw_png": tf.FixedLenSequenceFeature([], dtype=tf.string)
        })
    
    x = tf.map_fn(tf.image.decode_png, sequence_features['raw_png'], dtype=tf.uint8,
                back_prop=False, swap_memory=False, infer_shape=False)
    x = tf.cast(x, tf.float32)
    x /= 255
    x.set_shape(shape)
    x = tf.expand_dims(x[:,:,:,0], axis=3)
    
    return x, features['time_stamp']

In [7]:
from utils.transforms import *

sz = 128
nt = 10
stats = np.fromfile('stat.csv', sep=',')

class Slice(Transform):
    """ Return a slice of the images
    
    Arguments:
    The same as the built-in function slice
    """
    def __init__(self, *args, **kwargs):
        self.slice = slice(*args, **kwargs)
        super().__init__(TfmType.NO)
        
    def do_transform(self, x, is_y):
        return x[self.slice]
    
aug_tfms = [Slice(nt)]
tfms, _ = tfms_from_stats(stats, sz, aug_tfms=aug_tfms, crop_type=CropType.NO)
tfms

[<utils.transforms.Scale object at 0x7fb8c2839b38>, <__main__.Slice object at 0x7fb8c2823748>, <utils.transforms.Normalize object at 0x7fb8c28236d8>]

In [8]:
bs = 10

filenames = tf.placeholder(tf.string, name='tfrecords')
training_filenames = '../data/tfrecords/train_1.tfrecords'
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(parser_train).map(tfms).batch(bs).repeat()
# dataset = dataset.prefetch()
iterator = dataset.make_initializable_iterator()
iterator.initializer.run({filenames: training_filenames})
x, time_stamp = iterator.get_next()

## Refactoring PredNet

In [9]:
from models.prednet_refactored import PredNetCell, PredNet

# n_channels, im_height, im_width = (3, 128, 160)
n_channels, im_height, im_width = (1, sz, sz)
input_shape = (im_height, im_width, n_channels)
stack_sizes = (n_channels, 48, 96, 192)
R_stack_sizes = stack_sizes
A_filt_sizes = (3, 3, 3)
Ahat_filt_sizes = (3, 3, 3, 3)
R_filt_sizes = (3, 3, 3, 3)

layer_loss_weights = np.array([1., 0., 0., 0.])  # weighting for each layer in final loss; "L_0" model:  [1, 0, 0, 0], "L_all": [1, 0.1, 0.1, 0.1]
layer_loss_weights = np.expand_dims(layer_loss_weights, 1)
time_loss_weights = 1./ (nt - 1) * np.ones((nt,1))  # equally weight all timesteps except the first
time_loss_weights[0] = 0

prednet = PredNet(stack_sizes,
                  R_stack_sizes,
                  A_filt_sizes, 
                  Ahat_filt_sizes, 
                  R_filt_sizes)

ValueError: ('All cells must have a `call` method. received cells:', (1, 48, 96, 192))

In [ ]:
# inputs = tf.keras.layers.Input(shape=(nt,) + input_shape)
errors = prednet(x)  # errors will be (batch_size, nt, nb_layers)
# errors = prednet(x[0][0])  # errors will be (batch_size, nt, nb_layers)
errors_by_time = TimeDistributed(Dense(1, trainable=False), weights=[layer_loss_weights, np.zeros(1)], trainable=False)(errors)  # calculate weighted error by layer
errors_by_time = Flatten()(errors_by_time)  # will be (batch_size, nt)
final_errors = Dense(1, weights=[time_loss_weights, np.zeros(1)], trainable=False)(errors_by_time)  # weight errors by time
model = Model(inputs=inputs, outputs=final_errors)
model.compile(loss='mean_absolute_error', optimizer='adam')

In [88]:
model.fit(x, y, steps_per_epoch=10)

Epoch 1/1
10/10 [==============================] - 64s 6s/step - loss: 0.3643


## Putting it all together

### Data

In [11]:
from utils.transforms import *
sz = 128
nt = 10
bs = 4

num_gpus = 2

class Slice(Transform):
    """ Return a slice of the images
    
    Arguments:
    The same as the built-in function slice
    """
    def __init__(self, *args, **kwargs):
        self.slice = slice(*args, **kwargs)
        super().__init__(TfmType.NO)
        
    def do_transform(self, x, is_y):
        return x[self.slice]

aug_tfms = [Slice(nt)]

In [12]:
def input_fn(sz=128, nt=10, aug_tfms=aug_tfms,
             stats_fn='stat.csv', stats_sep=','):
    shape = (61, 501, 501, 3)
    def parser_train(serialized_example):
        features, sequence_features = tf.parse_single_sequence_example(
            serialized_example, context_features={
                'time_stamp': tf.FixedLenFeature([], tf.string),
            }, sequence_features={
                "raw_png": tf.FixedLenSequenceFeature([], dtype=tf.string)
            })

        x = tf.map_fn(tf.image.decode_png, sequence_features['raw_png'], dtype=tf.uint8,
                    back_prop=False, swap_memory=False, infer_shape=False)
        x = tf.cast(x, tf.float32)
        x /= 255
        x.set_shape(shape)
        x = tf.expand_dims(x[:,:,:,0], axis=3)

        return x, features['time_stamp']
    
    filenames = tf.placeholder(tf.string, name='tfrecords')
    
    stats = np.fromfile(stats_fn, sep=stats_sep)
    tfms, _ = tfms_from_stats(stats, sz, aug_tfms=aug_tfms, crop_type=CropType.NO)
    
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(parser_train).map(tfms).batch(bs).repeat()
    # dataset = dataset.prefetch()
    
    return dataset

### Model

In [13]:
from models.prednet_refactored import PredNetCell, PredNet

# n_channels, im_height, im_width = (3, 128, 160)
n_channels, im_height, im_width = (1, sz, sz)
input_shape = (im_height, im_width, n_channels)
stack_sizes = (n_channels, 48, 96, 192)
R_stack_sizes = stack_sizes
A_filt_sizes = (3, 3, 3)
Ahat_filt_sizes = (3, 3, 3, 3)
R_filt_sizes = (3, 3, 3, 3)

layer_loss_weights = np.array([1., 0., 0., 0.])  # weighting for each layer in final loss; "L_0" model:  [1, 0, 0, 0], "L_all": [1, 0.1, 0.1, 0.1]
layer_loss_weights = np.expand_dims(layer_loss_weights, 1)
time_loss_weights = 1./ (nt - 1) * np.ones((nt,1))  # equally weight all timesteps except the first
time_loss_weights[0] = 0

prednet_cell = PredNetCell(stack_sizes=stack_sizes,
                    R_stack_sizes=R_stack_sizes,
                    A_filt_sizes=A_filt_sizes,
                    Ahat_filt_sizes=Ahat_filt_sizes,
                    R_filt_sizes=R_filt_sizes)

prednet = PredNet(prednet_cell)

In [14]:
inputs = tf.keras.Input(shape=(nt,) + input_shape)
errors = prednet(inputs)  # errors will be (batch_size, nt, nb_layers)
errors_by_time = TimeDistributed(Dense(1, trainable=False), weights=[layer_loss_weights, np.zeros(1)], trainable=False)(errors)  # calculate weighted error by layer
errors_by_time = Flatten()(errors_by_time)  # will be (batch_size, nt)
final_errors = Dense(1, weights=[time_loss_weights, np.zeros(1)], trainable=False)(errors_by_time)  # weight errors by time
model = Model(inputs=inputs, outputs=final_errors)
# model.compile(loss='mean_absolute_error', optimizer='adam')
# model = tf.keras.utils.multi_gpu_model(model, gpus=num_gpus)
# model.compile(loss='mean_absolute_error', optimizer='adam')
model.compile(loss='mean_absolute_error', optimizer=tf.train.AdamOptimizer())

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 128, 128, 1)   0         
_________________________________________________________________
pred_net (PredNet)           (None, 10, 4)             6909818   
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 1)             5         
_________________________________________________________________
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 6,909,834
Trainable params: 6,909,818
Non-trainable params: 16
_________________________________________________________________


### save model

In [9]:
def input_fn(fns = ['../data/tfrecords/train_1.tfrecords'],
             sz=128, nt=10, aug_tfms=aug_tfms,
             stats_fn='stat.csv', stats_sep=','):
    dataset = tf.data.TFRecordDataset(fns)
    
    y = tf.zeros([bs, 1])
    def parser_train(serialized_example):
        # experimental. TODO: read only needed samples
        shape = (61, 501, 501, 3)
        context_features = {
                'time_stamp': tf.FixedLenFeature([], tf.string),
            }
        sequence_features = {
                "raw_png": tf.FixedLenSequenceFeature([], dtype=tf.string)
            }
        
        features, sequence_features = tf.parse_single_sequence_example(
            serialized_example, 
            context_features=context_features, 
            sequence_features=sequence_features)

        x = tf.map_fn(tf.image.decode_png, sequence_features['raw_png'], dtype=tf.uint8,
                    back_prop=False, swap_memory=False, infer_shape=False)
        x = tf.cast(x, tf.float32)
        x /= 255
        x.set_shape(shape)
        x = tf.expand_dims(x[:,:,:,0], axis=3)
        return x, y
    
    stats = np.fromfile(stats_fn, sep=stats_sep)
    tfms, _ = tfms_from_stats(stats, sz, aug_tfms=aug_tfms, crop_type=CropType.NO)
    
    dataset = dataset.map(parser_train)
    dataset = dataset.map(tfms)
    dataset = dataset.batch(bs)
    dataset = dataset.repeat()
    # dataset = dataset.prefetch()
    
#     return dataset
    
    y = tf.zeros([bs, 1])
    iterator = dataset.make_one_shot_iterator()
    x, _ = iterator.get_next()
    return x, y

In [ ]:
x, y = input_fn()

In [20]:
??keras.callbacks.TensorBoard(write_grads=True, write_images=True)

In [19]:
callbacks = [
    keras.callbacks.TensorBoard(write_grads=True, write_images=True)
]

In [18]:
model.fit(x, y, steps_per_epoch=10, callbacks=callbacks)

Epoch 1/1
10/10 [==============================] - 7s 723ms/step - loss: 0.1705


In [13]:
model.evaluate(x, y, steps=10)

10/10 [==============================] - 4s 444ms/step


0.28812453150749207

In [39]:
model.save('keras',overwrite=True)

In [40]:
custom_objects = {'PredNetCell': PredNetCell, 'PredNet': PredNet}
model = tf.keras.models.load_model('keras', custom_objects=custom_objects)

### Convert to estimator to leverage multiple GPUs

In [10]:
# multiple gpus
strategy = tf.contrib.distribute.MirroredStrategy(num_gpus=2)
session_config = tf.ConfigProto(log_device_placement=True, 
                                allow_soft_placement=True)
session_config.gpu_options.allow_growth = True
config = tf.estimator.RunConfig(train_distribute=strategy, session_config=session_config)

In [35]:
with tf.keras.utils.CustomObjectScope({'PredNetCell': PredNetCell, 'PredNet': PredNet}):
#     with tf.device('/gpu:0'):
    keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir='../data/models', config=config)
#     keras_estimator = tf.keras.estimator.model_to_estimator(keras_model=model, model_dir='../data/models')

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '../data/models', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
log_device_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.contrib.distribute.python.mirrored_strategy.MirroredStrategy object at 0x7efa66e380f0>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7efa65c59e10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [29]:
def input_fn(fns = ['../data/tfrecords/train_1.tfrecords'],
             sz=128, nt=10, aug_tfms=aug_tfms,
             stats_fn='stat.csv', stats_sep=','):
    dataset = tf.data.TFRecordDataset(fns)
    
    y = tf.zeros([bs, 1])
    def parser_train(serialized_example):
        # experimental. TODO: read only needed samples
        shape = (61, 501, 501, 3)
        context_features = {
                'time_stamp': tf.FixedLenFeature([], tf.string),
            }
        sequence_features = {
                "raw_png": tf.FixedLenSequenceFeature([], dtype=tf.string)
            }
        
        features, sequence_features = tf.parse_single_sequence_example(
            serialized_example, 
            context_features=context_features, 
            sequence_features=sequence_features)

        x = tf.map_fn(tf.image.decode_png, sequence_features['raw_png'], dtype=tf.uint8,
                    back_prop=False, swap_memory=False, infer_shape=False)
        x = tf.cast(x, tf.float32)
        x /= 255
        x.set_shape(shape)
        x = tf.expand_dims(x[:,:,:,0], axis=3)
        return x, y
    
    stats = np.fromfile(stats_fn, sep=stats_sep)
    tfms, _ = tfms_from_stats(stats, sz, aug_tfms=aug_tfms, crop_type=CropType.NO)
    
    dataset = dataset.map(parser_train)
    dataset = dataset.map(tfms)
    dataset = dataset.batch(bs)
    dataset = dataset.repeat()
    # dataset = dataset.prefetch()
    
    return dataset
    
    y = tf.zeros([bs, 1])
    iterator = dataset.make_one_shot_iterator()
    x, _ = iterator.get_next()
    return x, y

In [36]:
with tf.keras.utils.CustomObjectScope({'PredNetCell': PredNetCell, 'PredNet': PredNet}):
    keras_estimator.train(input_fn, steps=100, )

INFO:tensorflow:Device is available but not used by distribute strategy: /device:CPU:0
INFO:tensorflow:Configured nccl all-reduce.


[autoreload of tensorflow.python.keras.layers.core failed: Traceback (most recent call last):
  File "/home/ywx/data/miniconda3/envs/tf-nightly-gpu/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/ywx/data/miniconda3/envs/tf-nightly-gpu/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/home/ywx/data/miniconda3/envs/tf-nightly-gpu/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/ywx/data/miniconda3/envs/tf-nightly-gpu/lib/python3.6/importlib/__init__.py", line 147, in reload
    raise ImportError(msg.format(name), name=name)
ImportError: module tensorflow.python.keras.utils.multi_gpu_utils not in sys.modules
]


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:batch_all_reduce invoked for batches size = 46 with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='../data/models/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('../data/models/keras/keras_model.ckpt',)
INFO:tensorflow:Warm-starting variable: pred_net/layer_a_0/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: pred_net/layer_a_0/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: pred_net/layer_a_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: pred_net/layer_a_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-start

ValueError: Tensor tower_1/pred_net_1/Variable is not found in ../data/models/keras/keras_model.ckpt checkpoint {'training/TFOptimizer/beta2_power': [], 'training/TFOptimizer/beta1_power': [], 'time_distributed_2/kernel': [4, 1], 'pred_net/layer_o_3/kernel/Adam_1': [3, 3, 576, 192], 'pred_net/layer_o_3/kernel/Adam': [3, 3, 576, 192], 'pred_net/layer_o_3/bias/Adam_1': [192], 'pred_net/layer_o_3/bias/Adam': [192], 'pred_net/layer_o_2/kernel/Adam_1': [3, 3, 480, 96], 'pred_net/layer_o_2/bias/Adam_1': [96], 'pred_net/layer_o_1/kernel/Adam_1': [3, 3, 240, 48], 'pred_net/layer_o_1/kernel': [3, 3, 240, 48], 'pred_net/layer_o_1/bias/Adam': [48], 'pred_net/layer_o_0/kernel': [3, 3, 51, 1], 'pred_net/layer_o_0/bias': [1], 'pred_net/layer_i_3/bias/Adam_1': [192], 'pred_net/layer_o_2/bias/Adam': [96], 'pred_net/layer_i_3/bias': [192], 'pred_net/layer_i_2/kernel/Adam': [3, 3, 480, 96], 'pred_net/layer_i_2/kernel': [3, 3, 480, 96], 'pred_net/layer_i_2/bias': [96], 'pred_net/layer_i_1/kernel/Adam_1':